Se cargan las líbrerias

In [0]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegressionCV
from sklearn.metrics import accuracy_score



Se cargan los datos

In [30]:
data = pd.read_csv(
    "https://raw.githubusercontent.com/jdvelasq/datalabs/master/datasets/sms-spam.csv",
    sep = ',',
    thousands = None,
    decimal = '.',
    encoding='latin-1')
data

,type,text
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5569,spam,This is the 2nd time we have tried 2 contact u...
5570,ham,Will Ã¼ b going to esplanade fr home?
5571,ham,"Pity, * was in mood for that. So...any other s..."
5572,ham,The guy did some bitching but I acted like i'd...


Creamos una copia de los datos (mails) para operar sobre ella y luego eliminamos los afijos morfológicos de las palabras, dejando únicamente su raíz; ésto lo agregamos a una nueva columna.

In [31]:
mails=data.copy()
stemmer = PorterStemmer()

mails['stemmed'] = mails.text.apply(lambda x: ' '.join([stemmer.stem(w) for w in x.split() ]))

mails.head(10)


,type,text,stemmed
0,ham,"Go until jurong point, crazy.. Available only ...","Go until jurong point, crazy.. avail onli in b..."
1,ham,Ok lar... Joking wif u oni...,Ok lar... joke wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,free entri in 2 a wkli comp to win FA cup fina...
3,ham,U dun say so early hor... U c already then say...,U dun say so earli hor... U c alreadi then say...
4,ham,"Nah I don't think he goes to usf, he lives aro...","nah I don't think he goe to usf, he live aroun..."
5,spam,FreeMsg Hey there darling it's been 3 week's n...,freemsg hey there darl it' been 3 week' now an...
6,ham,Even my brother is not like to speak with me. ...,even my brother is not like to speak with me. ...
7,ham,As per your request 'Melle Melle (Oru Minnamin...,As per your request 'mell mell (oru minnaminun...
8,spam,WINNER!! As a valued network customer you have...,winner!! As a valu network custom you have bee...
9,spam,Had your mobile 11 months or more? U R entitle...,had your mobil 11 month or more? U R entitl to...


Hallamos la matriz de palabras del documento, ignorando aquellas palabras que tengan poca frecuencia (menores a 5) y convertimos todo a minuscula 

In [32]:
##
## Matriz de términos del documento
##
count_vect = CountVectorizer(
    analyzer='word',        # a nivel de palabra
    lowercase=True,         # convierte a minúsculas
    stop_words='english',   # stop_words en inglés
    min_df=5)               # ignora palabras con baja freq

##
## Aplica la función al texto
##
df_dtm = count_vect.fit_transform(mails.stemmed)

##
## Las filas contienen los mensajes
## y las clomunas los términos
##
df_dtm.shape

(5574, 1540)

Dividimos nuestros datos en entrenamiento y test, donde x es la matriz de terminos, y Y es la columna "type" de mails.
Luego aplicamos regresión logistica con cross-validation con LogisticRegressionCV quien aplica Kfold estratificado por default



In [35]:
X = df_dtm.toarray()
X_train, X_test, y_train, y_test = train_test_split(X, mails["type"], test_size=0.25)

model = LogisticRegressionCV(solver='liblinear', penalty='l1')
model.fit(X_train, y_train)
pred = model.predict(X_test)


0.9806312769010043

Ahora, podemos ver que porcentaje de palabras es capaz de acertar nuestro modelo

In [39]:
accuracy_score(y_test,pred)

0.9806312769010043